In [8]:
import heapq
import time
import random

class Noeud:
    def __init__(self, state, pred=None, g=0, h=0, move_count=0):
        self.state = state  # the state of the puzzle
        self.pred = pred  # predecessor node
        self.succ = []  # successors
        self.g = g  # g-value for A* (cost so far)
        self.h = h  # h-value for A* (heuristic estimate)
        self.move_count = move_count # Number of moves so far

    def getSucc(self, k=0):
        succ = []
        zero_index = self.state.index(0)
        n = int(len(self.state) ** 0.5)  # Determine grid size (n x n)
        row, col = divmod(zero_index, n)  # Get row and column of the zero tile

        # Regular sliding moves
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Possible moves: right, left, down, up

        for dr, dc in moves:
            new_row, new_col = row + dr, col + dc

            if 0 <= new_row < n and 0 <= new_col < n:  # Check if move is valid
                new_index = new_row * n + new_col
                new_state = self.state[:]
                new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
                succ.append(Noeud(new_state, pred=self))
                succ.append(Noeud(new_state, pred=self, move_count=self.move_count + 1))

        # Add swap moves if k-swap is allowed and move count % k == 0
        if k > 0 and self.move_count % k == 0 and self.move_count != 0:
            for i in range(len(self.state)):
                for j in range(i + 1, len(self.state)):
                    if i != zero_index and j != zero_index:  # Do not swap with the empty tile
                        new_state = self.state[:]
                        new_state[i], new_state[j] = new_state[j], new_state[i]
                        succ.append(Noeud(new_state, pred=self, move_count=self.move_count + 1))
                        
        return succ
    
    def addSuccessor(self, succ):
        self.succ.append(succ)
    
    def isSuccess(self):
        return self.state == sorted(self.state[:-1]) + [0]
        """
        for i in range(len(self.state)):
            if self.state[i] != (i + 1) % 9:
                return False
        return True
        """
    
    def getPred(self):
        return self.pred
    
    def akorotana(self, n):
        # Example implementation of the akorotana method (depending on what it's doing)
        # return random.shuffle(self)
        current_node = self
        for _ in range(n):
            current_node = random.choice(current_node.getSucc())
        return current_node
        #return self  # Returning self just for the sake of example

    def __lt__(self, other):
        # Define the comparison based on A* f = g + h
        return (self.g + self.h) < (other.g + other.h)
    
    def __str__(self):
        return str(self.state)  # Simple string representation of the state


def a_star(depart, k):
    closed_list = set()  # Closed set in A*
    open_list = []  # Priority Queue in Python (using heapq)
    heapq.heappush(open_list, depart)
    
    while open_list:
        current = heapq.heappop(open_list)
        
        if current.isSuccess():
            return current
        
        closed_list.add(tuple(current.state))
        
        for s in current.getSucc(k):
            if tuple(s.state) not in closed_list:
                # Set g and h for the successor node
                s.g = current.g + 1  # Assuming cost of 1 for each step
                s.h = heuristic(s)  # You'll need to define a heuristic function
                heapq.heappush(open_list, s)
    
    return None  # No path found


def heuristic(node):
    # Example heuristic: Manhattan distance (or any suitable heuristic for your problem)
    n = int(len(node.state) ** 0.5)
    goal = list(range(1, len(node.state))) + [0]
    distance = 0
    for i, value in enumerate(node.state):
        if value != 0:
            goal_index = goal.index(value)
            distance += abs(i // 3 - goal_index // 3) + abs(i % 3 - goal_index % 3)
    return distance


def main():
    n = 3
    k = 10
    
    nd = Noeud([1, 2, 3, 4, 5, 6, 7, 8, 0])

    nd = nd.akorotana(20)  # Shuffle the puzzle
    initial_state = nd
    print(f"Initialisation : {initial_state}")
   
    # Solve using A* search
    solution = a_star(nd, k)
    
    if solution:
        path = []
        while solution:
            path.append(solution)
            solution = solution.pred
        
        # Reverse path to get the start-to-goal order
        path.reverse()

        # Insert initial state like step 0
        path.insert(0, Noeud(initial_state.state))  
        
        for step, node in enumerate(path):
            print(f"Step {step}: {node}")  # Here, display or update GUI with the node (this is a placeholder)
            time.sleep(0.1)  # Sleep to simulate delay, replace with GUI update logic
    else:
        print("No solution found")


if __name__ == "__main__":
    main()

Initialisation : [1, 2, 3, 4, 0, 5, 7, 8, 6]
Step 0: [1, 2, 3, 4, 0, 5, 7, 8, 6]
Step 1: [1, 2, 3, 4, 5, 6, 7, 8, 0]
Step 2: [1, 2, 3, 4, 5, 6, 7, 0, 8]
Step 3: [1, 2, 3, 4, 5, 6, 7, 8, 0]
Step 4: [1, 2, 3, 4, 5, 0, 7, 8, 6]
Step 5: [1, 2, 0, 4, 5, 3, 7, 8, 6]
Step 6: [1, 2, 3, 4, 5, 0, 7, 8, 6]
Step 7: [1, 2, 3, 4, 5, 6, 7, 8, 0]
Step 8: [1, 2, 3, 4, 5, 0, 7, 8, 6]
Step 9: [1, 2, 3, 4, 0, 5, 7, 8, 6]
Step 10: [1, 2, 3, 4, 8, 5, 7, 0, 6]
Step 11: [1, 2, 3, 4, 8, 5, 0, 7, 6]
Step 12: [1, 2, 3, 4, 8, 5, 7, 0, 6]
Step 13: [1, 2, 3, 4, 8, 5, 0, 7, 6]
Step 14: [1, 2, 3, 0, 8, 5, 4, 7, 6]
Step 15: [1, 2, 3, 4, 8, 5, 0, 7, 6]
Step 16: [1, 2, 3, 4, 8, 5, 7, 0, 6]
Step 17: [1, 2, 3, 4, 8, 5, 0, 7, 6]
Step 18: [1, 2, 3, 4, 8, 5, 7, 0, 6]
Step 19: [1, 2, 3, 4, 0, 5, 7, 8, 6]
Step 20: [1, 2, 3, 0, 4, 5, 7, 8, 6]
Step 21: [1, 2, 3, 4, 0, 5, 7, 8, 6]
Step 22: [1, 2, 3, 4, 5, 0, 7, 8, 6]
Step 23: [1, 2, 3, 4, 5, 6, 7, 8, 0]
